<center> </center>

<center><font size=5 face="Helvetica" color=#EE4B2B><b>
Pytorch Tutorial: Build the Neural Network
</b></font></center>

<center><font face="Helvetica" size=3><b>Ang Chen</b></font></center>
<center><font face="Helvetica" size=3>July, 2024</font></center>

***

Neural networks comprise of layers/modules that perform operations on data.
The $\texttt{torch.nn}$ namespace provides all the building blocks you need to build your neural neetwork.
Every module in PyTorch subclasses the $\mathtt{nn.Module}$.
A neural network is a module itself that consists of other modules (layers).
This nested structure allows for building and managing complex architectures easily.

In the fowllowing sections, we will build a neural network to classify images in the FashionMNIST dataset.

In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

We want to be able to train our model on a hardware accelerator like the GPU or MPS, if available.
Let's check to see if $\texttt{torch.cuda}$ or $\texttt{torch.backend.mps}$ are available, otherwise we use the CPU.

In [7]:
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

In [8]:
print(f"Using {device} device.")

Using mps device.


# Define the Class

We define our neural network by subclassing $\texttt{nn.Module}$, and initialize the neural network layers in $\texttt{\_\_init\_\_}$.
Every $\texttt{nn.Module}$ subclass implements the operations on input data in the $\texttt{forward}$ method.

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of $\texttt{NeuralNetwork}$, and move it to the $\texttt{device}$, and print its structure.

In [11]:
model = NeuralNetwork().to(device=device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

To use the model. we pass it the input data.
This executes the model's $\texttt{forward}$, along with some background operations.
Do not call $\texttt{model.forward()}$ directly.

Calling the model on the imput returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output.
We get the prediction probabilities by passing it through an instantce of $\texttt{nn.Softmax}$ module.

In [16]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='mps:0')


## Model Layers

Let's break down the layers in the FashionMNIST model.
To illustrate it, we will take a sample minibatch of 3 images of size 28x28 
and see what happens to it as we pass it through the network.

In [19]:
input_image = torch.rand(3,28,28)
input_image.size()

torch.Size([3, 28, 28])